## Building RAG Chatbots with LangChain

Using LangChain, OpenAI and Pinecone vector DB, I build a chatbot capable of learning from the external world using Retrieval Augmented Generation (RAG).

I am using a dataset from a Llama2 ArXiv paper and related papers to provide context to the chatbot when it answers questions about the latest technologies and developments in Generative AI.  


Python libraries used
*   **langchain**: GenAI library, used to chain together different language components in the chatbot
*   **openai**: OpenAI Python client
*   **datasets**: Machine learning datasets, used as knowledge base for chatbot
*   **pinecone-client**: Pinecone Python client, use Pinecone API to store chatbot's knowledge base in a vector database



In [72]:
!pip install -qU langchain openai datasets tiktoken pinecone-client
!pip install -qU langchain_openai
!pip install -U langchain-community

When can prompt chatgpt using lists of messages. We can add new prompts by appending them to the existing list of messages.

In [93]:
import os
import openai
from langchain_openai import ChatOpenAI

openai.api_key = os.getenv("OPENAI_API_KEY")

os.environ['OPENAI_API_KEY'] = openai.api_key

chat = ChatOpenAI(
    openai_api_key=openai.api_key,
    model='gpt-3.5-turbo'
)

In [76]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpfu assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory")
]

res = chat.invoke(messages)
print(res.content)

prompt = HumanMessage(content="Why do physicists believe it can produce a unified theory?")

messages.append(prompt)

res = chat.invoke(messages)

print(res.content)


String theory is a theoretical framework in physics that attempts to explain the fundamental particles and forces in the universe in terms of one-dimensional strings rather than point-like particles. These strings vibrate at different frequencies, giving rise to different particles and forces. The theory aims to unify general relativity (which describes gravity) and quantum mechanics (which describes the other fundamental forces) into a single framework.

String theory is a complex and mathematically intricate subject that is still under active research and development. It has the potential to provide a unified description of all fundamental forces and particles in the universe, including gravity, electromagnetism, and the strong and weak nuclear forces. However, it also faces a number of challenges and criticisms, and many aspects of the theory remain speculative.

If you have any specific questions about string theory or would like more information on a particular aspect of the theor

### Hallucinations

The knowledge base of LLMs can be limited, because everything they know is learned during training. The LLM's worldview is entirely contained in the model's internal parameters. This is the chatbot's *parametric knowledge*.

As a result, LLMs have no knowledge about new developments such as the Llama 2 LLM and LLMChain:

In [77]:
messages.append(res)
prompt = HumanMessage(
    content="What is so special about Llama 2?"
)

messages.append(prompt)

res = chat.invoke(messages)

print(res.content)

I apologize but I'm not familiar with what "Llama 2" refers to. Can you provide more context or clarify your question so I can help you better?


In [78]:
messages.append(res)
prompt = HumanMessage(
    content="Tell me about LLMChain in LangChain?"
)

messages.append(prompt)

res = chat.invoke(messages)

print(res.content)

I'm sorry, but I'm not familiar with "LLMChain" or "LangChain." It's possible that these terms are related to specific projects or technologies that I have not encountered before. If you can provide more information or context, I'll do my best to assist you.


Another way of feeding knowledge in LLMs is using *source knowledge*, which is information that is fed into the LLM via the prompt. We can do this manually by adding a description from LangChain documentation

In [79]:
llmchain_info = [
    "A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.",
    "Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case.",
    "LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an api, but will also: (1) Be data-aware: connect a language model to other sources of data, (2) Be agentic: Allow a language model to interact with its environment. As such, the LangChain framework is designed with the objective in mind to enable those types of applications."
]

source_knowledge = "\n".join(llmchain_info)

query = "Can you tell me about the LLMChain in LangChain?"

augmented_prompt = f"""Using the contexts below, answer the query.

Contexts: {source_knowledge}

Query: {query}"""

prompt = HumanMessage(
    content=augmented_prompt
)
messages.append(prompt)

res = chat.invoke(messages)

messages.append(prompt)

res = chat.invoke(messages)

print(res.content)

The LLMChain is a fundamental component within the LangChain framework for developing applications powered by language models. It is the most common type of chain and consists of a PromptTemplate, a model (which can be either an LLM or a ChatModel), and an optional output parser. The LLMChain takes multiple input variables, utilizes the PromptTemplate to format them into a prompt, then passes that prompt to the model. Finally, it applies the OutputParser (if provided) to parse the output of the LLM into a final format. The LLMChain within LangChain plays a crucial role in leveraging language models to create powerful and differentiated applications that are data-aware and provide an agentic interaction with the environment.


### Importing the Data

We will be using the Hugging Face Dataset library to load context data. The dataset, "`jamescalam/llama-2-arxiv-papers`", contains a collection of ArXiv papers which serve as the external knowledge base the chatbot. Every entry in the dataset is a sample paragraph from a paper.

In [80]:
from datasets import load_dataset

dataset = load_dataset(
    "jamescalam/llama-2-arxiv-papers-chunked",
    split="train"
)

print(dataset[0])

{'doi': '1102.0183', 'chunk-id': '0', 'chunk': 'High-Performance Neural Networks\nfor Visual Object Classi\x0ccation\nDan C. Cire\x18 san, Ueli Meier, Jonathan Masci,\nLuca M. Gambardella and J\x7f urgen Schmidhuber\nTechnical Report No. IDSIA-01-11\nJanuary 2011\nIDSIA / USI-SUPSI\nDalle Molle Institute for Arti\x0ccial Intelligence\nGalleria 2, 6928 Manno, Switzerland\nIDSIA is a joint institute of both University of Lugano (USI) and University of Applied Sciences of Southern Switzerland (SUPSI),\nand was founded in 1988 by the Dalle Molle Foundation which promoted quality of life.\nThis work was partially supported by the Swiss Commission for Technology and Innovation (CTI), Project n. 9688.1 IFF:\nIntelligent Fill in Form.arXiv:1102.0183v1  [cs.AI]  1 Feb 2011\nTechnical Report No. IDSIA-01-11 1\nHigh-Performance Neural Networks\nfor Visual Object Classi\x0ccation\nDan C. Cire\x18 san, Ueli Meier, Jonathan Masci,\nLuca M. Gambardella and J\x7f urgen Schmidhuber\nJanuary 2011\nAbstr

### Building the Knowledge Base

Using Pinecone I set up an embedding model and vector database. I am using OpenAI's `text-embedding-ada-002` model for the embeddings with `dimension` set to `1536`.

In [81]:
from pinecone import Pinecone
from pinecone import ServerlessSpec
import time

pc_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(pc_api_key)
spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

index_name = 'llama-2-rag'
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

if index_name not in existing_indexes:
    # create index if it doesn't exist
    pc.create_index(
        index_name,
        dimension=1536,
        metric='dotproduct',
        spec=spec
    )

    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
      time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

By making a vector embedding for the dataset we can perform queries on semantic similarity rather than matching words. We use Pinecone DB since it uses approximate nearest neighbors to quickly query similar items

In [103]:
from langchain_community.embeddings import OpenAIEmbeddings
from tqdm import tqdm

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

os.environ['OPENAI_API_KEY'] = openai.api_key

data = dataset.to_pandas()

batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
  i_end = min(len(data), i+batch_size)
  batch = data.iloc[i:i_end]

  # generate unique ids for each chunk
  ids = [f"{x['doi']}-{x['chunk-id']}" for i, x in batch.iterrows()]
  # embed text
  texts = [x['chunk'] for _, x in batch.iterrows()]
  embeds = embed_model.embed_documents(texts)
  metadata = [
      {'text': x['chunk'],
       'source': x['source'],
       'title': x['title']} for i, x in batch.iterrows()
  ]
  # add to pinecone
  index.upsert(vectors=zip(ids, embeds, metadata))

print(index.describe_index_stats())


100%|██████████| 49/49 [01:50<00:00,  2.25s/it]

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4838}},
 'total_vector_count': 4838}


### Retrieval Augmented Generation
Now the vector embedding can be incorporated into the chatbot as a knowledge base. For example, we can query the index about relevant information on Llama 2

In [104]:
from langchain.vectorstores import Pinecone

text_field = "text"

vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

query = "What is so special about Llama 2?"

vectorstore.similarity_search(query, k=3)

<ipython-input-104-58d7a0d0e073>:5: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  vectorstore = Pinecone(
/usr/local/lib/python3.11/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


[Document(metadata={'source': 'http://arxiv.org/pdf/2307.09288', 'title': 'Llama 2: Open Foundation and Fine-Tuned Chat Models'}, page_content='Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang\nRoss Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang\nAngela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic\nSergey Edunov Thomas Scialom\x03\nGenAI, Meta\nAbstract\nIn this work, we develop and release Llama 2, a collection of pretrained and ﬁne-tuned\nlarge language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.\nOur ﬁne-tuned LLMs, called L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc , are optimized for dialogue use cases. Our\nmodels outperform open-source chat models on most benchmarks we tested, and based on\nourhumanevaluationsforhelpfulnessandsafety,maybeasuitablesubstituteforclosedsource models. We provide a detailed description of our approach to ﬁne-tu

The following function will connect the relevant results to the query prompt.

In [106]:
def augment_prompt(query: str):
  # return top 3 results from knowledge base
  results = vectorstore.similarity_search(query, k=3)
  source_knowledge = "\n".join([x.page_content for x in results])
  augmented_prompt = f"""Using the context below, answer the query.
  Contexts:
  {source_knowledge}

  Query: {query}"""
  return augmented_prompt

print(augment_prompt(query))

Using the context below, answer the query.
  Contexts:
  Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang
Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang
Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic
Sergey Edunov Thomas Scialom
GenAI, Meta
Abstract
In this work, we develop and release Llama 2, a collection of pretrained and ﬁne-tuned
large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.
Our ﬁne-tuned LLMs, called L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc , are optimized for dialogue use cases. Our
models outperform open-source chat models on most benchmarks we tested, and based on
ourhumanevaluationsforhelpfulnessandsafety,maybeasuitablesubstituteforclosedsource models. We provide a detailed description of our approach to ﬁne-tuning and safety
asChatGPT,BARD,andClaude. TheseclosedproductLLMsareheavilyﬁne-tunedtoalignwithhuman


In [107]:
prompt = HumanMessage(
    content=augment_prompt(query)
)
messages.append(prompt)

res = chat(messages)

print(res.content)

LLama 2 is a collection of pretrained and fine-tuned large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters. Their fine-tuned LLMs, specifically optimized for dialogue use cases, outperform open-source chat models on various benchmarks and have shown promising results in terms of helpfulness and safety based on human evaluations. These models may serve as a viable substitute for closed-source models in certain applications. The development and release of LLama 2 aim to provide models that demonstrate superior performance compared to existing open-source models and are competitive with closed-source models in terms of usability and safety.


If we ask about a new prompt chatgpt cannot respond with as much detail. It only remembers what was already said above.

In [108]:
prompt = HumanMessage(
    content="what safety measure were used in the development of Llama 2?"
)

res = chat(messages + [prompt])
print(res.content)

The development of Llama 2 focused on implementing safety measures to ensure the models were optimized for dialogue use cases and considered safe for deployment. The safety measures used in the development of Llama 2 included humane evaluations for helpfulness and safety as substitutes for closed-source models. Additionally, the fine-tuned LLMs in Llama 2 were designed to perform better than existing open-source models in terms of safety and helpfulness. These evaluations and optimizations were part of the strategy to enhance the usability and safety of the Llama 2 models.


We need to get new relevant source knowledge for the prompt.

In [109]:
prompt = HumanMessage(
    content=augment_prompt("what safety measure were used in the development of Llama 2?")
)
messages.append(prompt)

res = chat(messages)

print(res.content)

In the development of Llama 2, safety measures were taken to increase the safety of the models. These safety measures included using safety-specific data annotation and tuning, conducting red-teaming exercises, and employing iterative evaluations. Additionally, the researchers shared a thorough description of their fine-tuning methodology and their approach to improving LLM safety to encourage more responsible development of large language models.


In [ ]:
# delete index to save resources
pc.delete_index(index_name)